<a href="https://colab.research.google.com/github/denncraft/AI_ML_notes/blob/main/Deep_Learning_2023_chapter_3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

возвращаясь к примеру из chapter 2, где была написана модель  и класс NaiveDense

если его переписать, базирруясь на классе keras -> layer

In [5]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

# класс наследуется от класса `Layer`
class SimpleDense(keras.layers.Layer):
  def __init__(self, units, activation=None):
    super().__init__()
    self.units = units
    self.activation = activation

  # веса создаются в метоже build()
  def build(self, input_shape):
    input_dim = input_shape[-1]
    self.W = self.add_weight(shape=(input_dim, self.units), initializer="random_normal")
    self.b = self.add_weight(shape=(self.units,), initializer="zeros")

  # вычисления прямого прохода определяются в метоже call()
  def call(self, inputs):
    y = tf.matmul(inputs, self.W) + self.b
    if self.activation is not None:
      y = self.activation(y)
    return y


теперь создание слоя будет выглядеть так:

причем при создании слоя мы определяем только его выходной размер, а размерность входных тензоров слоя определит сам и создаст соответствующей размерности тензоры для хранения коэффициентов.
при этом мы даже не вызываем метод build(), он автоматически вызывается методом __call__.

In [7]:
my_dense = SimpleDense(units=32, activation=tf.nn.relu)
input_tensor = tf.ones(shape=(2,784)) # к примеру всего 2 элемента, но мы помним, что такая формы получалась после слоя flaten из (2, 28, 28)
output_tensor = my_dense(input_tensor)

print(output_tensor.shape)

(2, 32)


#сети бывают:

- сети с двумя ветвями (two-branches networks)
- многоголовые сети (multihead networks)
- входные блоки (inception blocks)

#оптимизаторы:
- SGD (с импульсом или без)
- RMSprop
- Adam
- Adagrad
- ...

#функции потерь:
- CategoricalCrossentropy
- SparseCategoricalCrossEntropy
- BinaryCrossentropy
- MeanSquareError
- KLDivergence
- CosineSimilitary
- ...

#метрики:
- CategicalAccuracy
- SparceCategoricalAccuracy
- BinaryAccuracy
- AUC
- Preccision
- Recall
- ...



In [9]:
from keras.datasets import mnist
(inputs, targets), (test_images, test_labels) = mnist.load_data()

In [11]:
inputs = inputs.reshape((60000, 784))
inputs = inputs.astype("float32") / 255

разделим данные на две части 30/70 для обучение/валидации

и перемешаем данные, чтобы вданные для обучения были не из начала или конца а вперемешку

обучим модель


In [ ]:
model = keras.Sequential([keras.layers.Dense(10)])
model.compile(optimizer=keras.optimizers.RMSprop(learning_rate=0.01),
              loss=keras.losses.MeanSquaredError(),
              metrics=[keras.metrics.BinaryAccuracy()])

indices_permutation = np.random.permutation(len(inputs))
shuffled_inputs = inputs[indices_permutation]
shuffled_targets = targets[indices_permutation]

num_validation_samples = int(0.3 * len(inputs))
val_inputs = shuffled_inputs[:num_validation_samples]
val_targets = shuffled_targets[:num_validation_samples]
training_inputs = shuffled_inputs[num_validation_samples:]
training_targets = shuffled_targets[num_validation_samples:]
model.fit(
    training_inputs,
    training_targets,
    epochs=50,
    batch_size=128,
    validation_data=(val_inputs, val_targets)
)